# A step-by-step guide on building game isolation agent with minimax, iterative deepning and alphabeta pruning 

### Create a Sample Game

In [1]:
from random import randint

if __name__ == "__main__":
    
    # from isolation.py get the Board class
    from isolation import Board
    from sample_players import *

    ## Setup
    # create an isolation board (by default 7x7) and some players
    player1 = RandomPlayer()
    player2 = GreedyPlayer()
    game = Board(player1, player2)

    # place player 1 on the board at row 2, column 3, then place player 2 on
    # the board at row 0, column 5; display the resulting board state.  Note
    # that .apply_move() changes the calling object
    
    # apply move and print the game board
    game.apply_move((2, 3))
    game.apply_move((0, 5))
    print(game.to_string())
    
    ## Play
    
    # players take turns moving on the board, so player1 should be next to move
    assert(player1 == game.active_player)

    # get a list of the legal moves available to the active player
    print(game.get_legal_moves())

    # get a successor of the current state by making a copy of the board and
    # applying a move. Notice that this does NOT change the calling object
    # (unlike .apply_move()).
    
    # make a new board as a branch of the old board, and assert not the same as before, then print both 
    new_game = game.forecast_move((1, 1))
    assert(new_game.to_string() != game.to_string())
    print("\nOld state:\n{}".format(game.to_string()))
    print("\nNew state:\n{}".format(new_game.to_string()))

    # play the remainder of the game automatically -- outcome can be "illegal
    # move" or "timeout"; it should _always_ be "illegal move" in this example
    
    # play the game, and get the winner, history, and outcome from 
    winner, history, outcome = game.play()
    print("\nWinner: {}\nOutcome: {}".format(winner, outcome))
    print(game.to_string())
    print("Move history:\n{!s}".format(history))

     0   1   2   3   4   5   6
0  |   |   |   |   |   | 2 |   | 
1  |   |   |   |   |   |   |   | 
2  |   |   |   | 1 |   |   |   | 
3  |   |   |   |   |   |   |   | 
4  |   |   |   |   |   |   |   | 
5  |   |   |   |   |   |   |   | 
6  |   |   |   |   |   |   |   | 

[(1, 5), (3, 5), (4, 4), (0, 2), (0, 4), (4, 2), (1, 1), (3, 1)]

Old state:
     0   1   2   3   4   5   6
0  |   |   |   |   |   | 2 |   | 
1  |   |   |   |   |   |   |   | 
2  |   |   |   | 1 |   |   |   | 
3  |   |   |   |   |   |   |   | 
4  |   |   |   |   |   |   |   | 
5  |   |   |   |   |   |   |   | 
6  |   |   |   |   |   |   |   | 


New state:
     0   1   2   3   4   5   6
0  |   |   |   |   |   | 2 |   | 
1  |   | 1 |   |   |   |   |   | 
2  |   |   |   | - |   |   |   | 
3  |   |   |   |   |   |   |   | 
4  |   |   |   |   |   |   |   | 
5  |   |   |   |   |   |   |   | 
6  |   |   |   |   |   |   |   | 


Winner: <sample_players.GreedyPlayer object at 0x7f2f007d1668>
Outcome: illegal move
     0   1   2 

### Our game agent

In [2]:
class SearchTimeout(Exception):
    """Subclass base exception for code clarity. """
    pass


def custom_score(game, player):
    """Calculate the heuristic value of a game state from the point of view
    of the given player.

    This should be the best heuristic function for your project submission.

    Note: this function should be called from within a Player instance as
    `self.score()` -- you should not need to call this function directly.

    Parameters
    ----------
    game : `isolation.Board`
        An instance of `isolation.Board` encoding the current state of the
        game (e.g., player locations and blocked cells).

    player : object
        A player instance in the current game (i.e., an object corresponding to
        one of the player objects `game.__player_1__` or `game.__player_2__`.)

    Returns
    -------
    float
        The heuristic value of the current game state to the specified player.
    """
    if game.is_loser(player):
        return -math.inf
    if game.is_winner(player):
        return math.inf
    
    player_moves = len(game.get_legal_moves(player))
    opponent_moves = len(game.get_legal_moves(game.get_opponent(player)))
    blank_spaces = game.get_blank_spaces()
    percent_board_occupied = int((len(blank_spaces)/(game.width * game.height)) * 100)
    
    if percent_board_occupied < 30 :
        return float(player_moves - 2*opponent_moves)
    else:
        return float(2*player_moves - opponent_moves)
    

    # list of locations that fall onto the walls of board 
#    walls = [
#        [(0, i) for i in range(game.width)],
#        [(i, 0) for i in range(game.height)],
#        [(game.width - 1, i) for i in range(game.width)],
#        [(i, game.height - 1) for i in range(game.height)]
#    ]
#
#    # list of corner locations of the board  
#    corners = [(0,0), (0,game.width-1), (game.height-1,0), (game.height-1,game.width-1)]
#    
#    
#
#    player_moves = len(game.get_legal_moves(player))
#    opponent_moves = len(game.get_legal_moves(game.get_opponent(player)))
#    
#    moves_wall = []
#    moves_corner = []
#    for move in game.get_legal_moves(player):
#        if move in walls:
#           moves_wall.append(move)
#        elif move in corners:
#           moves_corner.append(move)
#           
#    moves_wall_corner = len(moves_wall) + len(moves_corner)
#    better_moves= player_moves - moves_wall_corner
#    
#    

def custom_score_2(game, player):
    """Calculate the heuristic value of a game state from the point of view
    of the given player.

    Note: this function should be called from within a Player instance as
    `self.score()` -- you should not need to call this function directly.

    Parameters
    ----------
    game : `isolation.Board`
        An instance of `isolation.Board` encoding the current state of the
        game (e.g., player locations and blocked cells).

    player : object
        A player instance in the current game (i.e., an object corresponding to
        one of the player objects `game.__player_1__` or `game.__player_2__`.)

    Returns
    -------
    float
        The heuristic value of the current game state to the specified player.
    """
    if game.is_loser(player):
        return -math.inf
    if game.is_winner(player):
        return math.inf

    player_moves = len(game.get_legal_moves(player))
    opponent_moves = len(game.get_legal_moves(game.get_opponent(player)))
    return float(2*player_moves-opponent_moves)

def custom_score_3(game, player):
    """Calculate the heuristic value of a game state from the point of view
    of the given player.

    Note: this function should be called from within a Player instance as
    `self.score()` -- you should not need to call this function directly.

    Parameters
    ----------
    game : `isolation.Board`
        An instance of `isolation.Board` encoding the current state of the
        game (e.g., player locations and blocked cells).

    player : object
        A player instance in the current game (i.e., an object corresponding to
        one of the player objects `game.__player_1__` or `game.__player_2__`.)

    Returns
    -------
    float
        The heuristic value of the current game state to the specified player.
    """
    if game.is_loser(player):
        return -math.inf
    if game.is_winner(player):
        return math.inf

    player_moves = len(game.get_legal_moves(player))
    opponent_moves = len(game.get_legal_moves(game.get_opponent(player)))
    return float(player_moves-2*opponent_moves)



class IsolationPlayer:
    """Base class for minimax and alphabeta agents -- this class is never
    constructed or tested directly.
    ********************  DO NOT MODIFY THIS CLASS  ********************
    Parameters
    ----------
    search_depth : int (optional)
        A strictly positive integer (i.e., 1, 2, 3,...) for the number of
        layers in the game tree to explore for fixed-depth search. (i.e., a
        depth of one (1) would only explore the immediate sucessors of the
        current state.)
    score_fn : callable (optional)
        A function to use for heuristic evaluation of game states.
    timeout : float (optional)
        Time remaining (in milliseconds) when search is aborted. Should be a
        positive value large enough to allow the function to return before the
        timer expires.
    """
    def __init__(self, search_depth=30, score_fn=custom_score, timeout=35.):
        self.search_depth = search_depth
        self.score = score_fn
        self.time_left = None
        self.TIMER_THRESHOLD = timeout


class MinimaxPlayer(IsolationPlayer):
    """Game-playing agent that chooses a move using depth-limited minimax
    search. You must finish and test this player to make sure it properly uses
    minimax to return a good move before the search time limit expires.
    """


    def isTimeout(self):
        if self.time_left() < self.TIMER_THRESHOLD:
           raise SearchTimeout()

    def get_move(self, game, time_left):
        """Search for the best move from the available legal moves and return a
        result before the time limit expires.
        **************  YOU DO NOT NEED TO MODIFY THIS FUNCTION  *************
        For fixed-depth search, this function simply wraps the call to the
        minimax method, but this method provides a common interface for all
        Isolation agents, and you will replace it in the AlphaBetaPlayer with
        iterative deepening search.
        Parameters
        ----------
        game : `isolation.Board`
            An instance of `isolation.Board` encoding the current state of the
            game (e.g., player locations and blocked cells).
        time_left : callable
            A function that returns the number of milliseconds left in the
            current turn. Returning with any less than 0 ms remaining forfeits
            the game.
        Returns
        -------
        (int, int)
            Board coordinates corresponding to a legal move; may return
            (-1, -1) if there are no available legal moves.
        """
        self.time_left = time_left



        # Initialize the best move so that this function returns something
        # in case the search fails due to timeout
        best_move = (-1, -1)

        try:
            # The try/except block will automatically catch the exception
            # raised when the timer is about to expire.
            return self.minimax(game, self.search_depth)

        except SearchTimeout:
            pass  # Handle any actions required after timeout as needed

        # Return the best move from the last completed search iteration
        return best_move


    def maxvalue(self, game, max_depth, curr_depth):
        """
        Search for the branch with the highest score value.  Return that value.
        Parameters
        ---------------
        game : `isolation.Board`
            An instance of `isolation.Board` encoding the current state of the
            game (e.g., player locations and blocked cells).
        max_depth: int
            The maximum number of plies to check (this remains the same for
            Minimax, and will increase on each run in the AlphaBetaPlayer 
            which uses iterative deepening)
        curr_depth: int
            The current depth being checked (incremented on each call until it reaches
            max_depth)
        Returns
        -----------------
        val : int
            The score for the best move available (the maximum given all the legal moves available)
        """

        if self.time_left() < self.TIMER_THRESHOLD:
           raise SearchTimeout()

        # Reached the depth we wanted to consider. Return the score for that
        #   move (based on the heuristics in improved_score, or custom_score)
        if (max_depth==curr_depth):
            return self.score(game, self)
        else:

            # Increase depth counter for when we next call minvalue
            curr_depth+=1
            moves = game.get_legal_moves(self)
            if len(moves)==0:
                # No legals moves left, player would lose if we chose this branch
                return float("-inf")

            best_move = moves[0]
            maxv=float("-inf")

            # Consider each move. If it returns a higher predicted value than
            #   what was previously stored in best_move, replace best_move and maxv
            for m in moves:
                val = self.minvalue(game.forecast_move(m), max_depth, curr_depth)
                if (val>maxv):
                    maxv = val
                    best_move = m    
            return maxv

    

    def minvalue(self, game, max_depth, curr_depth):
        """
        Search for the branch with the highest score value.  Return that value.
        Parameters
        ---------------
        game : `isolation.Board`
            An instance of `isolation.Board` encoding the current state of the
            game (e.g., player locations and blocked cells).
        max_depth: int
            The maximum number of plies to check (this remains the same for
            Minimax, and will increase on each run in the AlphaBetaPlayer 
            which uses iterative deepening)
        curr_depth: int
            The current depth being checked (incremented on each call until it reaches
            max_depth)
        Returns
        -----------------
        val : int
            The score for the best move available (the maximum given all the legal moves available)
        """
        if self.time_left() < self.TIMER_THRESHOLD:
            raise SearchTimeout()

        # Reached the depth we wanted to consider. Return the score for that
        #   move (based on the heuristics in improved_score, or custom_score)
        if (max_depth==curr_depth):
            return self.score(game, self)
        else:
            # Increase depth counter for when we next call maxvalue
            curr_depth+=1

            moves = game.get_legal_moves()
            if len(moves)==0:
                #no legal moves left for opponent, opponent would lose on this branch
                return float("+inf")

            best_move = moves[0]
            minv=float("inf")

            # Consider each move. If it returns a lower predicted value than
            #   what was previously stored in best_move, replace best_move and minv            
            for m in moves:
                val = self.maxvalue(game.forecast_move(m), max_depth, curr_depth)
                if (val<minv):
                    minv = val
                    best_move = m     
            return minv



    def minimax(self, game, depth):
        """Implement depth-limited minimax search algorithm as described in
        the lectures.
        This should be a modified version of MINIMAX-DECISION in the AIMA text.
        https://github.com/aimacode/aima-pseudocode/blob/master/md/Minimax-Decision.md
        **********************************************************************
            You MAY add additional methods to this class, or define helper
                 functions to implement the required functionality.
        **********************************************************************
        Parameters
        ----------
        game : isolation.Board
            An instance of the Isolation game `Board` class representing the
            current game state
        depth : int
            Depth is an integer representing the maximum number of plies to
            search in the game tree before aborting
        Returns
        -------
        (int, int)
            The board coordinates of the best move found in the current search;
            (-1, -1) if there are no legal moves
        Notes
        -----
            (1) You MUST use the `self.score()` method for board evaluation
                to pass the project tests; you cannot call any other evaluation
                function directly.
            (2) If you use any helper functions (e.g., as shown in the AIMA
                pseudocode) then you must copy the timer check into the top of
                each helper function or else your agent will timeout during
                testing.
        """

  
        if self.time_left() < self.TIMER_THRESHOLD:
            raise SearchTimeout()


        moves = game.get_legal_moves()

        if len(moves)==0:
            best_move = (-1, -1)   # no moves left, forfeit game

        else:              
            best_move = moves[0]
            maxval=float("-inf")

            # We're not doing iterative deepening, so we put the try/except around 
            #  testing each move. If we run out of time, we return the best move that
            #  we've found so far.

            for m in moves:
                try:
                    val = self.minvalue(game.forecast_move(m), depth, 1)
            
                except SearchTimeout:
                    return best_move
                
                if val>maxval:
                    maxval = val
                    best_move = m    


        return best_move



class AlphaBetaPlayer(IsolationPlayer):
    """Game-playing agent that chooses a move using iterative deepening minimax
    search with alpha-beta pruning. You must finish and test this player to
    make sure it returns a good move before the search time limit expires.
    """

    def isTimeout(self):
        if self.time_left() < self.TIMER_THRESHOLD:
           raise SearchTimeout()

    def get_move(self, game, time_left):
        """Search for the best move from the available legal moves and return a
        result before the time limit expires.
        Modify the get_move() method from the MinimaxPlayer class to implement
        iterative deepening search instead of fixed-depth search.
        **********************************************************************
        NOTE: If time_left() < 0 when this function returns, the agent will
              forfeit the game due to timeout. You must return _before_ the
              timer reaches 0.
        **********************************************************************
        Parameters
        ----------
        game : `isolation.Board`
            An instance of `isolation.Board` encoding the current state of the
            game (e.g., player locations and blocked cells).
        time_left : callable
            A function that returns the number of milliseconds left in the
            current turn. Returning with any less than 0 ms remaining forfeits
            the game.
        Returns
        -------
        (int, int)
            Board coordinates corresponding to a legal move; may return
            (-1, -1) if there are no available legal moves.
        """
        self.time_left = time_left


       # Initialize the best move so that this function returns something
        # in case the search fails due to timeout
        best_move = (-1, -1)
        depth = 1

        # Test if first move of the game -- if so pick the center square
        if len(game.get_blank_spaces()) == game.width * game.height:
            return (3,3)


        while depth <= self.search_depth:   
            try:
            # The try/except block will automatically catch the exception
            # raised when the timer is about to expire.
                best_move = self.alphabeta(game, depth)
                depth+=1

            except SearchTimeout:
#                print("Timeout- spaces remaining: ", len(game.get_blank_spaces()))
                return best_move


#        print("Spaces remaining: ", len(game.get_blank_spaces()))
        # Return the best move from the last completed search iteration
        return best_move



    def maxvalue(self, game, alpha, beta, max_depth, curr_depth):
        """
        Search for the branch with the highest score value.  Return that value.
        Parameters
        ---------------
        game : `isolation.Board`
            An instance of `isolation.Board` encoding the current state of the
            game (e.g., player locations and blocked cells).
        alpha : float
            The low end of alpha-beta pruning -- while you are in minvalue() function,
            if minv is lower than this, you know this branch won't get considered in the
            maxvalue node above it, so you can stop testing
            
        beta : float
            The high end of alpha-beta pruning -- while you are in maxvalue() function, 
            if maxv is higher than this, you know this branch won't get considered in the 
            minvalue node above it, so you can stop testing 
        max_depth: int
            The maximum number of plies to check (this remains the same for
            Minimax, and will increase on each run in the AlphaBetaPlayer 
            which uses iterative deepening)
        curr_depth: int
            The current depth being checked (incremented on each call until it reaches
            max_depth)
        Returns
        -----------------
        val : float
            The score for the best move available (the maximum given all the legal moves available)
        """

        if self.time_left() < self.TIMER_THRESHOLD:
           raise SearchTimeout()


        # Reached the depth we wanted to consider. Return the score for that
        #   move (based on the heuristics in improved_score, or custom_score)
        if (max_depth==curr_depth):
            return self.score(game, self)

        else:
            # Increase depth counter for when we next call minvalue
            curr_depth+=1
            moves = game.get_legal_moves(self)
            if len(moves)==0:
                # No legals moves left
                return float("-inf")
            
            maxv = float("-inf")

            # Consider each move. If it returns a higher predicted value than
            #   what was previously stored in best_move, replace maxv

            for m in moves:
                val = self.minvalue(game.forecast_move(m), alpha, beta, max_depth, curr_depth)
                if val>maxv:
                    maxv = val

                # Check if maxv is more than beta - this means it is more than a neighboring node, and the
                # minvalue above will never choose this branch. You can stop searching this branch and 
                # return upwards.
                if maxv >= beta:
                    return maxv

                # Need to continue searching this branch. If maxv is higher than the previously saved alpha,
                #  set alpha to be equal to this new max
                else:
                    alpha = max(alpha, maxv) 

            return maxv


    def minvalue(self, game, alpha, beta, max_depth, curr_depth):
        """
        Search for the branch with the highest score value.  Return that value.
        Parameters
        ---------------
        game : `isolation.Board`
            An instance of `isolation.Board` encoding the current state of the
            game (e.g., player locations and blocked cells).
        alpha : float
            The low end of alpha-beta pruning -- while you are in minvalue() function,
            if minv is lower than this, you know this branch won't get considered in the
            maxvalue node above it, so you can stop testing
            
        beta : float
            The high end of alpha-beta pruning -- while you are in maxvalue() function, 
            if maxv is higher than this, you know this branch won't get considered in the 
            minvalue node above it, so you can stop testing 
        max_depth: int
            The maximum number of plies to check (this remains the same for
            Minimax, and will increase on each run in the AlphaBetaPlayer 
            which uses iterative deepening)
        curr_depth: int
            The current depth being checked (incremented on each call until it reaches
            max_depth)
        Returns
        -----------------
        val : float
            The score for the best move available (the maximum given all the legal moves available)
        """

        if self.time_left() < self.TIMER_THRESHOLD:
            raise SearchTimeout()

        # Reached the depth we wanted to consider. Return the score for that
        #   move (based on the heuristics in improved_score, or custom_score)
        if (max_depth==curr_depth):
            return self.score(game, self)
        else:
            # Increase depth counter for when we next call maxvalue
            curr_depth+=1

            moves = game.get_legal_moves()
            if len(moves)==0:
                #no legal moves left for opponent, opponent would lose on this branch
                return float("+inf")

            minv = float("inf")

            # Consider each move. If a given move returns a lower predicted value than
            #   what was previously found, replace minv
            for m in moves:
                val = self.maxvalue(game.forecast_move(m), alpha, beta, max_depth, curr_depth)
                if val < minv:
                    minv = val

                # Check if minv is less than alpha - this means it is less than a neighboring node, and the
                # maxvalue above will never choose this branch. You can stop searching this branch and 
                # return upwards.
                if minv <= alpha:
                    return minv

                # Need to continue searching this branch. If minv is lower than the previously saved beta,
                #  set beta to be equal to this new min
                else:
                    beta = min(beta, minv)
            return minv


    def alphabeta(self, game, depth, alpha=float("-inf"), beta=float("inf")):
        """Implement depth-limited minimax search with alpha-beta pruning as
        described in the lectures.
        This should be a modified version of ALPHA-BETA-SEARCH in the AIMA text
        https://github.com/aimacode/aima-pseudocode/blob/master/md/Alpha-Beta-Search.md
        **********************************************************************
            You MAY add additional methods to this class, or define helper
                 functions to implement the required functionality.
        **********************************************************************
        Parameters
        ----------
        game : isolation.Board
            An instance of the Isolation game `Board` class representing the
            current game state
        depth : int
            Depth is an integer representing the maximum number of plies to
            search in the game tree before aborting
        alpha : float
            Alpha limits the lower bound of search on minimizing layers
        beta : float
            Beta limits the upper bound of search on maximizing layers
        Returns
        -------
        (int, int)
            The board coordinates of the best move found in the current search;
            (-1, -1) if there are no legal moves
        Notes
        -----
            (1) You MUST use the `self.score()` method for board evaluation
                to pass the project tests; you cannot call any other evaluation
                function directly.
            (2) If you use any helper functions (e.g., as shown in the AIMA
                pseudocode) then you must copy the timer check into the top of
                each helper function or else your agent will timeout during
                testing.
        """
        if self.time_left() < self.TIMER_THRESHOLD:
            raise SearchTimeout()


        moves = game.get_legal_moves()
        if len(moves)==0:    # no legal moves available, forfeit the game
            return (-1, -1)
             
        best_move = moves[0]
        maxval=float("-inf")

        # Consider each available move, and select the move with the highest
        #   score (returned from searching the tree "depth" nodes down)
        #   Each time this alphabeta() function is called, depth will be one
        #   higher, and the search will be that much deeper

        for m in moves:
            val = self.minvalue(game.forecast_move(m), alpha, beta, depth, 1)
                
            if val>maxval:
                maxval = val
                best_move = m    

            # Increase alpha to the max value already found - this way minvalue() 
            #  can stop searching if it knows it will return a value lower than
            #  a max that was already found
            alpha = max(alpha, maxval)

        return best_move

### Tournament

In [4]:
import itertools
import random
import warnings

from collections import namedtuple

from isolation import Board
from sample_players import (RandomPlayer, open_move_score,
                            improved_score, center_score)
from game_agent import (MinimaxPlayer, AlphaBetaPlayer, custom_score,
                        custom_score_2, custom_score_3)

NUM_MATCHES = 10  # number of matches against each opponent
TIME_LIMIT = 100  # number of milliseconds before timeout

DESCRIPTION = """
This script evaluates the performance of the custom_score evaluation
function against a baseline agent using alpha-beta search and iterative
deepening (ID) called `AB_Improved`. The three `AB_Custom` agents use
ID and alpha-beta search with the custom_score functions defined in
game_agent.py.
"""

Agent = namedtuple("Agent", ["player", "name"])


def play_round(cpu_agent, test_agents, win_counts, num_matches):
    """Compare the test agents to the cpu agent in "fair" matches.

    "Fair" matches use random starting locations and force the agents to
    play as both first and second player to control for advantages resulting
    from choosing better opening moves or having first initiative to move.
    """
    timeout_count = 0
    forfeit_count = 0
    for _ in range(num_matches):

        games = sum([[Board(cpu_agent.player, agent.player),
                      Board(agent.player, cpu_agent.player)]
                    for agent in test_agents], [])

        # initialize all games with a random move and response
        for _ in range(2):
            move = random.choice(games[0].get_legal_moves())
            for game in games:
                game.apply_move(move)

        # play all games and tally the results
        for game in games:
            winner, _, termination = game.play(time_limit=TIME_LIMIT)
            win_counts[winner] += 1

            if termination == "timeout":
                timeout_count += 1
            elif termination == "forfeit":
                forfeit_count += 1

    return timeout_count, forfeit_count


def update(total_wins, wins):
    for player in total_wins:
        total_wins[player] += wins[player]
    return total_wins


def play_matches(cpu_agents, test_agents, num_matches):
    """Play matches between the test agent and each cpu_agent individually. """
    total_wins = {agent.player: 0 for agent in test_agents}
    total_timeouts = 0.
    total_forfeits = 0.
    total_matches = 2 * num_matches * len(cpu_agents)

    print("\n{:^9}{:^13}".format("Match #", "Opponent") + ''.join(['{:^13}'.format(x[1].name) for x in enumerate(test_agents)]))
    print("{:^9}{:^13} ".format("", "") +  ' '.join(['{:^5}| {:^5}'.format("Won", "Lost") for x in enumerate(test_agents)]))

    for idx, agent in enumerate(cpu_agents):
        wins = {key: 0 for (key, value) in test_agents}
        wins[agent.player] = 0

        print("{!s:^9}{:^13}".format(idx + 1, agent.name), end="", flush=True)

        counts = play_round(agent, test_agents, wins, num_matches)
        total_timeouts += counts[0]
        total_forfeits += counts[1]
        total_wins = update(total_wins, wins)
        _total = 2 * num_matches
        round_totals = sum([[wins[agent.player], _total - wins[agent.player]]
                            for agent in test_agents], [])
        print(' ' + ' '.join([
            '{:^5}| {:^5}'.format(
                round_totals[i],round_totals[i+1]
            ) for i in range(0, len(round_totals), 2)
        ]))

    print("-" * 74)
    print('{:^9}{:^13}'.format("", "Win Rate:") +
        ''.join([
            '{:^13}'.format(
                "{:.1f}%".format(100 * total_wins[x[1].player] / total_matches)
            ) for x in enumerate(test_agents)
    ]))

    if total_timeouts:
        print(("\nThere were {} timeouts during the tournament -- make sure " +
               "your agent handles search timeout correctly, and consider " +
               "increasing the timeout margin for your agent.\n").format(
            total_timeouts))
    if total_forfeits:
        print(("\nYour ID search forfeited {} games while there were still " +
               "legal moves available to play.\n").format(total_forfeits))


def main():

    # Define two agents to compare -- these agents will play from the same
    # starting position against the same adversaries in the tournament
    test_agents = [
        Agent(AlphaBetaPlayer(score_fn=improved_score), "AB_Improved"),
        Agent(AlphaBetaPlayer(score_fn=custom_score), "AB_Custom"),
        Agent(AlphaBetaPlayer(score_fn=custom_score_2), "AB_Custom_2"),
        Agent(AlphaBetaPlayer(score_fn=custom_score_3), "AB_Custom_3")
    ]

    # Define a collection of agents to compete against the test agents
    cpu_agents = [
        Agent(RandomPlayer(), "Random"),
        Agent(MinimaxPlayer(score_fn=open_move_score), "MM_Open"),
        Agent(MinimaxPlayer(score_fn=center_score), "MM_Center"),
        Agent(MinimaxPlayer(score_fn=improved_score), "MM_Improved"),
        Agent(AlphaBetaPlayer(score_fn=open_move_score), "AB_Open"),
        Agent(AlphaBetaPlayer(score_fn=center_score), "AB_Center"),
        Agent(AlphaBetaPlayer(score_fn=improved_score), "AB_Improved")
    ]

    print(DESCRIPTION)
    print("{:^74}".format("*************************"))
    print("{:^74}".format("Playing Matches"))
    print("{:^74}".format("*************************"))
    play_matches(cpu_agents, test_agents, NUM_MATCHES)


if __name__ == "__main__":
    main()



This script evaluates the performance of the custom_score evaluation
function against a baseline agent using alpha-beta search and iterative
deepening (ID) called `AB_Improved`. The three `AB_Custom` agents use
ID and alpha-beta search with the custom_score functions defined in
game_agent.py.

                        *************************                         
                             Playing Matches                              
                        *************************                         

 Match #   Opponent    AB_Improved   AB_Custom   AB_Custom_2  AB_Custom_3 
                        Won | Lost   Won | Lost   Won | Lost   Won | Lost 
    1       Random      19  |   1    19  |   1    20  |   0    20  |   0  
    2       MM_Open     14  |   6    13  |   7    18  |   2    17  |   3  
    3      MM_Center    16  |   4    13  |   7    13  |   7    13  |   7  
    4     MM_Improved   17  |   3    17  |   3    15  |   5    14  |   6  
    5       AB_Open     10  